---
# Imports

In [1]:
import numpy as np
import pandas as pd

---
# Data Import

In [ ]:
sensors = pd.read_csv('data/sensor.csv')

sensors = sensors[['SensorId', 'Position']]
sensors.head(3)

,SensorId,Position
0,007f2b03-94e6-47b3-9e3e-44273354acd5,"42.055622821657266,21.305010875644516"
1,01440b05-255d-4764-be87-bdf135f32289,"42.027618975073544,21.38741970062256"
2,01cf1cec-bf2d-41b3-8cd5-e8bd720f01b4,"41.993498532663594,21.44513139126384"


In [57]:
len(sensors)

176

---
# Data Preparation

In [58]:
sensors = sensors[sensors['SensorId'] != 'sensor_dev_78844_374'] # Belgija
sensors = sensors[sensors['SensorId'] != 'ef8fbcf0-e04e-4d15-ab3c-a625a2f9245d'] # Belgija

In [59]:
sensors['x'] = sensors['Position'].apply(lambda x: float(x.split(',')[0]))
sensors['y'] = sensors['Position'].apply(lambda x: float(x.split(',')[1]))

## Haversine distance

In [60]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = 6367 * c
    return km

## Sensor Class

In [ ]:
class Sensor:
    def __init__(self, x, y, id):
        self.x = x
        self.y = y
        self.id = id

        sensors_closest = []

    def get_distance(self, other_sensor):
        
        return haversine(self.x, self.y, other_sensor.x, other_sensor.y)
    
    def generate_closest(self, sensors, limit=5):
        assert limit > 0, "Limit must be greater than 0"
        
        self.sensors_closest = sorted(sensors, key=lambda x: self.get_distance(x))[:limit]

## Generate Sensor relations

In [ ]:
sensors_list = []
for row in sensors.itertuples():
    id = row.SensorId
    x = row.x
    y = row.y

    sensor = Sensor(x, y, id)
    sensors_list.append(sensor)

for idx, sensor in enumerate(sensors_list):
    sensor.generate_closest(sensors_list[:idx] + sensors_list[idx+1:])

In [63]:
new_sensors = []

for sensor in sensors_list:
    current_sensor = {
        'SensorId': sensor.id,
        'x': sensor.x,
        'y': sensor.y,
    }

    for idx, closest_sensor in enumerate(sensor.sensors_closest):
        current_sensor[f'closest_{idx+1}'] = closest_sensor.id
        current_sensor[f'distance_{idx+1}'] = sensor.get_distance(closest_sensor)

    new_sensors.append(current_sensor)

new_sensors_df = pd.DataFrame(new_sensors)

In [ ]:
new_sensors_df.head(3)

,SensorId,x,y,closest_1,distance_1,closest_2,distance_2,closest_3,distance_3,closest_4,distance_4,closest_5,distance_5
0,007f2b03-94e6-47b3-9e3e-44273354acd5,42.055623,21.305011,sensor_dev_84687_383,2.231466,sensor_dev_81888_631,3.479312,d1023aea-a20b-4ecd-845c-645410aa0ce7,4.197724,d46ddb51-9b1d-4590-b2ef-d3761d0898cc,4.262663,sensor_dev_78662_610,5.293686
1,01440b05-255d-4764-be87-bdf135f32289,42.027619,21.387420,sensor_dev_42668_227,0.063932,f3c9933e-0077-4408-8a0a-3392e91e1f23,0.774751,fefbf9e0-ff44-4b85-b968-2af046c4f4dc,1.569683,59c15198-7ba8-45af-aac4-5b3d1560fd81,1.956990,sensor_dev_60237_141,2.098843
2,01cf1cec-bf2d-41b3-8cd5-e8bd720f01b4,41.993499,21.445131,1005,0.727585,68538f26-664b-430b-a30e-162ad38c32b7,0.771036,sensor_dev_82724_611,0.909676,sensor_dev_80225_313,0.918557,a6ee3048-4533-4c64-bb9f-c6b55cf8f468,0.923554
3,07b58ccf-7faa-4f0a-a10a-e7b485d52ffe,42.073047,21.449465,3e2465de-c2c2-4473-9c62-7113265debd9,1.197101,sensor_dev_79190_458,2.292451,d42147ab-9f8c-4816-964b-62e6a82d6492,3.283515,7e78f939-b68e-4294-ab57-8fd607ecd050,4.262226,sensor_dev_80193_378,4.499972
4,0a058579-12c9-47be-971b-607198002d3b,41.993972,21.426850,sensor_dev_82685_644,0.122801,1000,0.313541,sensor_dev_82683_723,0.386332,d23d40cf-ea46-4791-99d5-ab0874246448,0.440092,e7a05c01-1d5c-479a-a5a5-419f28cebeef,0.446859
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,sensor_dev_83413_356,41.990784,21.407276,ece5f5f1-a572-4b6d-b5ce-54080e6b5845,0.528686,66710fdc-cdfc-4bbe-93a8-7e796fb8a88d,0.622337,7b9efe93-d604-4d6c-b2da-04c0b8882292,0.662169,3568aa20-235a-408c-861b-279c9f4d7709,0.693354,sensor_dev_61184_835,1.002492
170,sensor_dev_84687_383,42.040000,21.322000,sensor_dev_81888_631,1.786683,d46ddb51-9b1d-4590-b2ef-d3761d0898cc,2.086494,007f2b03-94e6-47b3-9e3e-44273354acd5,2.231466,d1023aea-a20b-4ecd-845c-645410aa0ce7,2.898461,sensor_dev_78662_610,3.107884
171,sensor_dev_84891_599,42.072000,21.394000,c38dabe0-8631-4f1a-b3ed-4014dacce39c,1.368329,d42147ab-9f8c-4816-964b-62e6a82d6492,2.642721,5346db83-fbf0-48c4-8714-1d896ca2f5e8,2.881572,1f966a58-fa92-4f41-b87c-786d4db58a74,3.110275,2d77519e-8c61-4d8e-87a6-d15126ea80c3,3.141246
172,sensor_dev_84941_208,42.002176,21.498149,sensor_dev_79771_603,0.329599,5de2a490-05f1-4f33-927e-a7e6f5664b72,0.341772,661b61de-e128-4966-ad32-22924713d1d5,1.053987,11888f3a-bc5e-4a0c-9f27-702984decedf,1.349722,sensor_dev_79773_456,1.573478
